## Day 3

You arrive at the airport only to realize that you grabbed your North Pole Credentials instead of your passport. While these documents are extremely similar, North Pole Credentials aren't issued by a country and therefore aren't actually valid documentation for travel in most of the world.

It seems like you're not the only one having problems, though; a very long line has formed for the automatic passport scanners, and the delay could upset your travel itinerary.

Due to some questionable network security, you realize you might be able to solve both of these problems at the same time.

The automatic passport scanners are slow because they're having trouble detecting which passports have all required fields. The expected fields are as follows:

- byr (Birth Year)
- iyr (Issue Year)
- eyr (Expiration Year)
- hgt (Height)
- hcl (Hair Color)
- ecl (Eye Color)
- pid (Passport ID)
- cid (Country ID)

Passport data is validated in batch files (your puzzle input). Each passport is represented as a sequence of key:value pairs separated by spaces or newlines. Passports are separated by blank lines.

Here is an example batch file containing four passports:

ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in

The first passport is valid - all eight fields are present. The second passport is invalid - it is missing hgt (the Height field).

The third passport is interesting; the only missing field is cid, so it looks like data from North Pole Credentials, not a passport at all! Surely, nobody would mind if you made the system temporarily ignore missing cid fields. Treat this "passport" as valid.

The fourth passport is missing two fields, cid and byr. Missing cid is fine, but missing any other field is not, so this passport is invalid.

According to the above rules, your improved system would report 2 valid passports.

Count the number of valid passports - those that have all required fields. Treat cid as optional. In your batch file, how many passports are valid?

In [1]:
# import pandas
import pandas as pd
import numpy as np

In [3]:
# Open file
f = open("Day4.txt", "r")

# Initialize our dataframe
passports = pd.DataFrame(columns=['byr','iyr','eyr','hgt','hcl','ecl','pid','cid'])

passport_num=0
passports.loc[passport_num] = [False for i in range(8)]

for line in f:    
    if line=="\n":
        # we can expect another passport
        passport_num+=1
        passports.loc[passport_num] = [False for i in range(8)]
    else:
        # we continue to process current password
        # split the string based on colon
        elements = line.split(" ")
        for element in elements:
            element = element.strip('\n')
            code, value = element.split(':')
            passports[code][passport_num]=True
        
        # Always set cid=True
        passports['cid'][passport_num]=True
            
# Close file
f.close()

In [4]:
passports

,byr,iyr,eyr,hgt,hcl,ecl,pid,cid
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...
285,True,True,True,True,True,True,True,True
286,True,True,True,True,False,True,True,True
287,True,True,True,True,True,True,True,True
288,False,True,False,True,True,True,True,True


In [5]:
passports['num_fields'] = passports.sum(axis=1)

In [6]:
(passports['num_fields']==8).sum()

237

The line is moving more quickly now, but you overhear airport security talking about how passports with invalid data are getting through. Better add some data validation, quick!

You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:

- byr (Birth Year) - four digits; at least 1920 and at most 2002.
- iyr (Issue Year) - four digits; at least 2010 and at most 2020.
- eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
- hgt (Height) - a number followed by either cm or in:
    - If cm, the number must be at least 150 and at most 193.
    - If in, the number must be at least 59 and at most 76.
- hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
- ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
- pid (Passport ID) - a nine-digit number, including leading zeroes.
- cid (Country ID) - ignored, missing or not.

Your job is to count the passports where all required fields are both present and valid according to the above rules. Here are some example values:

byr valid:   2002
byr invalid: 2003

hgt valid:   60in
hgt valid:   190cm
hgt invalid: 190in
hgt invalid: 190

hcl valid:   #123abc
hcl invalid: #123abz
hcl invalid: 123abc

ecl valid:   brn
ecl invalid: wat

pid valid:   000000001
pid invalid: 0123456789
Here are some invalid passports:

eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
Here are some valid passports:

pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719

Count the number of valid passports - those that have all required fields and valid values. Continue to treat cid as optional. In your batch file, how many passports are valid?



In [33]:
def val_hgt(hgt):
    """true if a number followed by either cm or in:
        If cm, the number must be at least 150 and at most 193.
        If in, the number must be at least 59 and at most 76."""
    if (hgt.find('cm')>=0):
        hgt = hgt.strip('cm').strip(' ').strip('.')
        if ('150'<=hgt<='193'):
            return True
    elif (hgt.find('in')>=0):
        hgt = hgt.strip('in').strip(' ').strip('.')
        if ('59'<=hgt<='76'):
            return True
    return False

In [11]:
def val_byr(byr):
    """true if four digits; at least 1920 and at most 2002"""
    if (len(byr)==4 and ('1920'<=byr<='2002')):
        return True
    else:
        return False

In [15]:
def val_iyr(iyr):
    """true if four digits; at least 2010 and at most 2020."""
    if (len(iyr)==4 and ('2010'<=iyr<='2020')):
        return True
    else:
        return False

In [25]:
def val_eyr(eyr):
    """true if four digits; at least 2020 and at most 2030."""
    if (len(eyr)==4 and ('2020'<=eyr<='2030')):
        return True
    else:
        return False

In [33]:
def val_hgt(hgt):
    """true if a number followed by either cm or in:
        If cm, the number must be at least 150 and at most 193.
        If in, the number must be at least 59 and at most 76."""
    if (hgt.find('cm')>=0):
        hgt = hgt.strip('cm').strip(' ').strip('.')
        if ('150'<=hgt<='193'):
            return True
    elif (hgt.find('in')>=0):
        hgt = hgt.strip('in').strip(' ').strip('.')
        if ('59'<=hgt<='76'):
            return True
    return False

In [60]:
def val_hcl(hcl):
    """true if a # followed by exactly six characters 0-9 or a-f"""
    if len(hcl)!=7:
        return False
    if hcl[0]!="#":
        return False
    for i in hcl[1:7]:
        if not (i.isnumeric() or i.islower()):
            return False
    
    # Passed every test!
    return True

In [71]:
def val_ecl(ecl):
    """true if exactly one of: amb blu brn gry grn hzl oth"""
    return ecl in ['amb','blu','blu','brn','gry','grn','hzl','oth']

In [76]:
def val_pid(pid):
    """true if a nine-digit number, including leading zeroes."""
    return (len(pid)==9 and pid.isnumeric())

In [92]:
# Open file
f = open("Day4.txt", "r")

# Initialize our dataframe
passports = pd.DataFrame(columns=['byr','iyr','eyr','hgt','hcl','ecl','pid','cid'])

passport_num=0
passports.loc[passport_num] = [False for i in range(8)]

for line in f:    
    if line=="\n":
        # we can expect another passport
        passport_num+=1
        passports.loc[passport_num] = [False for i in range(8)]
    else:
        # we continue to process current password
        # split the string based on colon
        elements = line.split(" ")
        for element in elements:
            element = element.strip('\n')
            code, value = element.split(':')
            if code=='byr':
                passports[code][passport_num]=val_byr(value)
            elif code=='iyr':
                passports[code][passport_num]=val_iyr(value) 
            elif code=='eyr':
                passports[code][passport_num]=val_eyr(value)
            elif code=='hgt':
                passports[code][passport_num]=val_hgt(value)
            elif code=='hcl':
                passports[code][passport_num]=val_hcl(value)
            elif code=='ecl':
                passports[code][passport_num]=val_ecl(value)
            elif code=='pid':
                passports[code][passport_num]=val_pid(value)
        # Always set cid=True
        passports['cid'][passport_num]=True
            
# Close file
f.close()

In [93]:
passports

,byr,iyr,eyr,hgt,hcl,ecl,pid,cid
0,True,True,True,True,True,True,True,True
1,True,True,True,False,True,True,True,True
2,True,True,True,True,True,True,True,True
3,False,True,False,False,False,True,False,True
4,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...
285,False,False,False,True,False,False,True,True
286,True,True,False,False,False,False,False,True
287,True,True,True,True,True,True,True,True
288,False,False,False,True,True,False,True,True


In [94]:
passports['num_fields'] = passports.sum(axis=1)

In [95]:
(passports['num_fields']==8).sum()

172